In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [10]:
data = pd.read_excel(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\7-8上交.xlsx", sheet_name='医院基础信息核对 （上交版本）')
df = data[data['科室']=='N'].reset_index().drop(columns='index')
df['医院名称'] = df['医院名称'].astype('string')
df

,医院名称,医院系数,医院编号,科室,name\ncross,address\ncross,医院属性1,医院属性2,分级,分等,...,对应主院编码,是否查找科室,是否有科室信息,无科室信息的原因,信息是否完全,是否反馈,备注,区域信息是否正确,辅助,Unnamed: 32
0,伊春市带岭区人民医院,0.7500,230725002,N,1.0,NaN,公立,综合,二级,甲,...,NaN,Y,N,已注销,Y,NaN,NaN,1.0,公立综合二级甲黑龙江省大庆市带岭区迎宾路95号,230725002.0
1,杜蒙县妇幼保健医院,0.5000,230624314,N,NaN,NaN,公立,专科,一级,甲,...,NaN,Y,N,重复医院,Y,NaN,NaN,1.0,公立综合一级甲黑龙江省大庆市杜尔伯特县北街,230624314.0
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,0.0938,230624313,N,NaN,NaN,公立,专科,二级,NaN,...,NaN,Y,N,重复医院,N,NaN,NaN,1.0,公立专科二级黑龙江省大庆市杜尔伯特县泰康镇哈萨尔路乌日根街交通巷,230624313.0
3,大庆市萨尔图区陈立军内科诊所,0.0313,410724004,N,1.0,NaN,民营,NaN,未定级,NaN,...,NaN,Y,N,未找到,N,NaN,NaN,1.0,民营未评级黑龙江省大庆市新村6-01-3-101,410724004.0
4,大庆市红岗区三九综合门诊,0.0313,230605005,N,1.0,NaN,民营,NaN,未定级,NaN,...,NaN,Y,N,重复医院,N,NaN,NaN,1.0,民营未评级黑龙江省大庆市黑龙江省大庆市红岗区红城国际E栋1号,230605005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,中国人民武装警察部队吉林省总队医院,0.2500,220103998,N,NaN,NaN,公立,综合,三级,NaN,...,NaN,N,N,NaN,N,NaN,NaN,NaN,公立综合三级吉林省长春市吉林省长春市农安南街766号,220103998.0
1817,吉林大学白求恩第一医院,0.0313,220104293,N,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N,Y,NaN,N,NaN,NaN,NaN,吉林省长春市,220104293.0
1818,九台区人民医院,0.0313,220113009,N,NaN,NaN,NaN,综合,NaN,NaN,...,NaN,N,Y,NaN,N,NaN,NaN,NaN,吉林省长春市,220113009.0
1819,长春中医药大学第二附属医院,0.0313,220103960,N,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N,N,NaN,N,NaN,NaN,NaN,吉林省长春市,220103960.0


In [11]:
def resolve(url, headers_list):
    for i in range(5):  # 循环去请求网站
        headers = np.random.choice(headers_list)
        response = requests.get(url, headers=headers, timeout=20)
        if response.status_code == 200:
            break
    return response

In [12]:
# 因为每个界面必定有p_tag以及b_tag, 当爬虫出错时换headers重新爬取界面信息
def find_b_tag(url, headers, p_tag):
    try:
        b_tag = p_tag.find("b")
        return b_tag
    except:
        response = resolve(url, headers_list)
        content = response.text
        soup = BeautifulSoup(content, 'html.parser')
        p_tag = soup.find("p", class_="result")
        if p_tag is None:
            return find_b_tag(url, headers, p_tag)
        else:
            return p_tag.find("b")

In [13]:
headers_list = [
       {
        'user-agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 SE 2.X MetaSr 1.0'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36 Core/1.94.202.400 QQBrowser/11.9.5355.400'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Mobile Safari/537.36'
    }
]
headers = np.random.choice(headers_list)

In [14]:
base_url = 'https://so.99.com.cn/search.php?s=relevance&proj=yyk&f=_all&q='
yiyuan = []
hospital_link = []

for i in np.arange(len(df["医院名称"])): 
    # index 209,607,809,824,825,829,952,961,968,983,1010,1036,1215,1216,1434,1435,1543,1711,1712,1797,1805,1806,1821,2016,2018,
    # 2333,2355,2417,2428,2532,2533,2534,2555,2560,2583,2946,2947,2973,3163,3487,3562,3671,3674,3676,3677,3678,3679,3820,3823,
    # 4345,4451,4452,4465,4466,4467,4468,4578,4597,4711,4740 --> raise error when searching on the website
    if any(char in df["医院名称"][i] for char in ['-', '/', '(', ')']):
        print(i)
        continue
    url = f'{base_url}{df["医院名称"][i]}'
    response = requests.get(url, headers=headers)
    content = response.text
    soup = BeautifulSoup(content, 'html.parser')
    p_tag = soup.find("p", class_="result")
    b_tag = find_b_tag(url, headers, p_tag)
    result = b_tag.get_text()
    if result !='0':
        link = soup.find('div', class_='yy-cont').find('a')
        if link:
            hospital_link.append(link['href'])
        else:
            hospital_link.append('NA')
    else:
        hospital_link.append('NA')
    yiyuan.append(df["医院名称"][i])
new_df = pd.DataFrame()
new_df['hospital'] = yiyuan
new_df['url'] = hospital_link
#new_df.to_excel('99health-new.xlsx',index=False)

1658


In [17]:
new_df.to_excel('99health-new.xlsx',index=False)
df = pd.read_excel('99health-new.xlsx')
df

,hospital,url
0,伊春市带岭区人民医院,https://yyk.99.com.cn/longfeng/85065/
1,杜蒙县妇幼保健医院,https://yyk.99.com.cn/daqing/97867/
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,https://yyk.99.com.cn/duerbote/101574/
3,大庆市萨尔图区陈立军内科诊所,https://yyk.99.com.cn/saertu/85059/
4,大庆市红岗区三九综合门诊,https://yyk.99.com.cn/daqing/85093/
...,...,...
1815,中国人民武装警察部队吉林省总队医院,https://yyk.99.com.cn/kuanchengqu/84243/
1816,吉林大学白求恩第一医院,NaN
1817,九台区人民医院,https://yyk.99.com.cn/jiutai/84317/
1818,长春中医药大学第二附属医院,NaN


In [22]:
df_drop=df.dropna()

In [25]:
df_drop.reset_index(drop=True, inplace=True)

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 假设 df_drop 已经包含了你想处理的数据，且已经删除了包含 NaN 的行
# 例如：df_drop = df_cleaned （上面清理后的 DataFrame）

# 初始化列表以保存数据
hospital_name = []
labels = []
address = []
keshi = []
yiyuan = []

# 可能需要调整开始的索引为之前出错的第一个索引，比如 64
start_index = 0  # 调整为你之前错误后的第一个索引
end_index = len(df_drop['hospital'])  # 继续到数据集的结束

for i in np.arange(start_index, end_index):
    url = df_drop['url'][i]
    response = requests.get(url, headers=headers)
    content = response.text
    soup = BeautifulSoup(content, 'html.parser')
    div_class = soup.find("div", class_ = "wrap_title")
    
    if div_class:
        # 医院名称
        hospital_name.append(div_class.find("b").get_text())
        # 标签
        labels.append([j.get_text() for j in div_class.find_all('span')])
        # 地址
        i_tag = soup.find('i', class_='wrap_yydz')
        if i_tag and i_tag.find_parent('em'):
            address.append(i_tag.find_parent('em').find_next_sibling('span').get_text())
        else:
            address.append('NA')
        # 科室信息
        div_tag = soup.find("div", class_="left_title")
        if div_tag and div_tag.get_text() == "医院科室":
            dept_texts = []
            next_div = div_tag.find_next_sibling('div')
            for dd in next_div.find_all('dd'):
                dept_texts.extend([a.get_text() for a in dd.find_all('a')])
            keshi.append(", ".join(dept_texts))
        else:
            keshi.append('NA')
    else:
        hospital_name.append('NA')
        labels.append('NA')
        address.append('NA')
        keshi.append('NA')
    
    # 每次循环，都添加当前医院的名称
    yiyuan.append(df_drop['hospital'][i])

# 创建 DataFrame
new_df = pd.DataFrame({
    'hospital': yiyuan,
    'hospital_name': hospital_name,  # 确保这些字段也添加到 DataFrame 中
    'labels': labels,
    'address': address,
    'keshi': keshi
})

new_df.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-keshi.csv", index=False)


In [42]:
new_df.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-keshi.csv", index=False, encoding='utf-8-sig')

In [29]:
new_df2 = pd.DataFrame()
new_df2['hospital'] = df_drop['hospital']
new_df2['other name'] = hospital_name
new_df2['labels'] = labels
new_df2['address'] = address
new_df2['url'] = df['url']
new_df2.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-other-info.csv",index=False)

In [43]:
new_df2.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-other-info.csv", index=False, encoding='utf-8-sig')

In [39]:
df1

,hospital,hospital_name,labels,address,keshi
0,伊春市带岭区人民医院,伊春市带岭区人民医院,"['二级甲等', '综合医院', '公立']",带岭区迎宾路95号,"内科, 外科"
1,杜蒙县妇幼保健医院,杜蒙县妇幼保健医院,"['一级甲等', '综合医院', '公立']",杜尔伯特县北街,"内科, 外科"
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,杜尔伯特蒙古族自治县妇幼保健计划生育服务...,"['二级医院', '专科医院', '公立']",杜尔伯特县泰康镇哈萨尔路乌日根街交通巷,"内科, 外科"
3,大庆市萨尔图区陈立军内科诊所,大庆市萨尔图区陈立军内科诊所,"['未评级', '其他', '民营']",新村6-01-3-101,NaN
4,大庆市红岗区三九综合门诊,大庆市红岗区三九综合门诊,"['未评级', '其他', '民营']",黑龙江省大庆市红岗区红城国际E栋1号,"内科, 外科"
...,...,...,...,...,...
1627,长春中医药大学附属医院妇产诊疗中心,长春中医药大学附属医院妇产诊疗中心,"['三级甲等', '综合医院', '公立']",长春市经济技术开发区深圳街2号,NaN
1628,长春中医药大学附属消化病医院,长春中医药大学附属消化病医院,"['未知', '其他', '其他']",临河街3828号,NaN
1629,中国人民武装警察部队吉林省总队医院,中国人民武装警察部队吉林省总队医院,"['三级医院', '综合医院', '公立']",吉林省长春市农安南街766号,"呼吸内科, 心血管内科, 消化内科, 神经内科, 烧伤整形科, 外二科, 口腔科, 传染科,..."
1630,九台区人民医院,长春市九台区人民医院,"['二级医院', '综合医院', '公立']",吉林省长春市九台区福星大街100号,"消化内科, 神经外科, 普外科, 妇产科, 传染病, 中医科, 放射科, 病理科, 麻醉科,..."


In [32]:
df1 = pd.read_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-keshi.csv")
df2 = pd.read_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-other-info.csv")
df2

,hospital,other name,labels,address,url
0,伊春市带岭区人民医院,伊春市带岭区人民医院,"['二级甲等', '综合医院', '公立']",带岭区迎宾路95号,https://yyk.99.com.cn/longfeng/85065/
1,杜蒙县妇幼保健医院,杜蒙县妇幼保健医院,"['一级甲等', '综合医院', '公立']",杜尔伯特县北街,https://yyk.99.com.cn/daqing/97867/
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,杜尔伯特蒙古族自治县妇幼保健计划生育服务...,"['二级医院', '专科医院', '公立']",杜尔伯特县泰康镇哈萨尔路乌日根街交通巷,https://yyk.99.com.cn/duerbote/101574/
3,大庆市萨尔图区陈立军内科诊所,大庆市萨尔图区陈立军内科诊所,"['未评级', '其他', '民营']",新村6-01-3-101,https://yyk.99.com.cn/saertu/85059/
4,大庆市红岗区三九综合门诊,大庆市红岗区三九综合门诊,"['未评级', '其他', '民营']",黑龙江省大庆市红岗区红城国际E栋1号,https://yyk.99.com.cn/daqing/85093/
...,...,...,...,...,...
1627,长春中医药大学附属医院妇产诊疗中心,长春中医药大学附属医院妇产诊疗中心,"['三级甲等', '综合医院', '公立']",长春市经济技术开发区深圳街2号,https://yyk.99.com.cn/chaoyang2/84189/
1628,长春中医药大学附属消化病医院,长春中医药大学附属消化病医院,"['未知', '其他', '其他']",临河街3828号,https://yyk.99.com.cn/chaoyang2/84146/
1629,中国人民武装警察部队吉林省总队医院,中国人民武装警察部队吉林省总队医院,"['三级医院', '综合医院', '公立']",吉林省长春市农安南街766号,https://yyk.99.com.cn/chaoyang2/84185/
1630,九台区人民医院,长春市九台区人民医院,"['二级医院', '综合医院', '公立']",吉林省长春市九台区福星大街100号,https://yyk.99.com.cn/chaoyang2/84193/


In [3]:
import pandas as pd
df1 = pd.read_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-keshi.csv")
df2 = pd.read_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-other-info.csv")
df2

,hospital,other name,labels,address,url
0,伊春市带岭区人民医院,伊春市带岭区人民医院,"['二级甲等', '综合医院', '公立']",带岭区迎宾路95号,https://yyk.99.com.cn/longfeng/85065/
1,杜蒙县妇幼保健医院,杜蒙县妇幼保健医院,"['一级甲等', '综合医院', '公立']",杜尔伯特县北街,https://yyk.99.com.cn/daqing/97867/
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,杜尔伯特蒙古族自治县妇幼保健计划生育服务...,"['二级医院', '专科医院', '公立']",杜尔伯特县泰康镇哈萨尔路乌日根街交通巷,https://yyk.99.com.cn/duerbote/101574/
3,大庆市萨尔图区陈立军内科诊所,大庆市萨尔图区陈立军内科诊所,"['未评级', '其他', '民营']",新村6-01-3-101,https://yyk.99.com.cn/saertu/85059/
4,大庆市红岗区三九综合门诊,大庆市红岗区三九综合门诊,"['未评级', '其他', '民营']",黑龙江省大庆市红岗区红城国际E栋1号,https://yyk.99.com.cn/daqing/85093/
...,...,...,...,...,...
1627,长春中医药大学附属医院妇产诊疗中心,长春中医药大学附属医院妇产诊疗中心,"['三级甲等', '综合医院', '公立']",长春市经济技术开发区深圳街2号,https://yyk.99.com.cn/chaoyang2/84189/
1628,长春中医药大学附属消化病医院,长春中医药大学附属消化病医院,"['未知', '其他', '其他']",临河街3828号,https://yyk.99.com.cn/chaoyang2/84146/
1629,中国人民武装警察部队吉林省总队医院,中国人民武装警察部队吉林省总队医院,"['三级医院', '综合医院', '公立']",吉林省长春市农安南街766号,https://yyk.99.com.cn/chaoyang2/84185/
1630,九台区人民医院,长春市九台区人民医院,"['二级医院', '综合医院', '公立']",吉林省长春市九台区福星大街100号,https://yyk.99.com.cn/chaoyang2/84193/


In [4]:
def get_level(label):
    if pd.isna(label):
        return 'NA'
    if '未评级' in label:
        return '未评级'
    elif '三' in label:
        return '三级'
    elif '二' in label:
        return '二级'
    elif '一' in label:
        return '一级'
    else:
        return 'NA'
        
def get_level2(label):
    if pd.isna(label):
        return 'NA'
    if '甲' in label:
        return '甲等'
    elif '乙' in label:
        return '乙等'
    elif '丙' in label:
        return '丙等'
    else:
        return 'NA'

def get_level3(label):
    if pd.isna(label):
        return 'NA'
    if '公立' in label:
        return '公立'
    elif '民营' in label:
        return '民营'
    elif '合资' in label:
        return '合资'
    else:
        return 'NA'

def get_level4(label):
    if pd.isna(label):
        return 'NA'
    if '综合' in label:
        return '综合医院'
    elif '专科' in label:
        return '专科医院'
    elif '妇幼保健' in label:
        return '妇幼保健院'
    elif '男科' in label:
        return '男科医院'
    elif '中医' in label:
        return '中医医院'
    elif '中西医' in label:
        return '中西医结合医院'
    elif '肿瘤' in label:
        return '肿瘤医院'
    elif '医保定点' in label:
        return '医保定点医院'
    else:
        return 'NA'

In [5]:
df2['分级'] = df2['labels'].apply(lambda x: get_level(x))
df2['分等'] = df2['labels'].apply(lambda x: get_level2(x))
df2['经营模式'] = df2['labels'].apply(lambda x: get_level3(x))
df2['类别'] = df2['labels'].apply(lambda x: get_level4(x))
df2

,hospital,other name,labels,address,url,分级,分等,经营模式,类别
0,伊春市带岭区人民医院,伊春市带岭区人民医院,"['二级甲等', '综合医院', '公立']",带岭区迎宾路95号,https://yyk.99.com.cn/longfeng/85065/,二级,甲等,公立,综合医院
1,杜蒙县妇幼保健医院,杜蒙县妇幼保健医院,"['一级甲等', '综合医院', '公立']",杜尔伯特县北街,https://yyk.99.com.cn/daqing/97867/,一级,甲等,公立,综合医院
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,杜尔伯特蒙古族自治县妇幼保健计划生育服务...,"['二级医院', '专科医院', '公立']",杜尔伯特县泰康镇哈萨尔路乌日根街交通巷,https://yyk.99.com.cn/duerbote/101574/,二级,NA,公立,专科医院
3,大庆市萨尔图区陈立军内科诊所,大庆市萨尔图区陈立军内科诊所,"['未评级', '其他', '民营']",新村6-01-3-101,https://yyk.99.com.cn/saertu/85059/,未评级,NA,民营,NA
4,大庆市红岗区三九综合门诊,大庆市红岗区三九综合门诊,"['未评级', '其他', '民营']",黑龙江省大庆市红岗区红城国际E栋1号,https://yyk.99.com.cn/daqing/85093/,未评级,NA,民营,NA
...,...,...,...,...,...,...,...,...,...
1627,长春中医药大学附属医院妇产诊疗中心,长春中医药大学附属医院妇产诊疗中心,"['三级甲等', '综合医院', '公立']",长春市经济技术开发区深圳街2号,https://yyk.99.com.cn/chaoyang2/84189/,三级,甲等,公立,综合医院
1628,长春中医药大学附属消化病医院,长春中医药大学附属消化病医院,"['未知', '其他', '其他']",临河街3828号,https://yyk.99.com.cn/chaoyang2/84146/,NA,NA,NA,NA
1629,中国人民武装警察部队吉林省总队医院,中国人民武装警察部队吉林省总队医院,"['三级医院', '综合医院', '公立']",吉林省长春市农安南街766号,https://yyk.99.com.cn/chaoyang2/84185/,三级,NA,公立,综合医院
1630,九台区人民医院,长春市九台区人民医院,"['二级医院', '综合医院', '公立']",吉林省长春市九台区福星大街100号,https://yyk.99.com.cn/chaoyang2/84193/,二级,NA,公立,综合医院


In [6]:
df2.drop(columns='labels', inplace=True)

In [18]:
merged = pd.merge(df1, df2, on ='hospital', how='left')
#merged = merged.iloc[:,[0,2,1,5,6,7,8,3,4]]
merged = merged.iloc[:,[0,1,10,11,8,9,4,8,9]]
merged.columns = ["医院名称", "别名", "医院属性1", "医院属性2", "分级","分等", "科室", "地址", "URL"]
merged=merged[["医院名称",  "医院属性1", "医院属性2", "分级","分等","科室"]]
merged

,医院名称,医院属性1,医院属性2,分级,分等,科室
0,伊春市带岭区人民医院,公立,综合医院,二级,甲等,"内科, 外科"
1,杜蒙县妇幼保健医院,公立,综合医院,一级,甲等,"内科, 外科"
2,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,公立,专科医院,二级,NA,"内科, 外科"
3,大庆市萨尔图区陈立军内科诊所,民营,NA,未评级,NA,NaN
4,大庆市红岗区三九综合门诊,民营,NA,未评级,NA,"内科, 外科"
...,...,...,...,...,...,...
1627,长春中医药大学附属医院妇产诊疗中心,公立,综合医院,三级,甲等,NaN
1628,长春中医药大学附属消化病医院,NA,NA,NA,NA,NaN
1629,中国人民武装警察部队吉林省总队医院,公立,综合医院,三级,NA,"呼吸内科, 心血管内科, 消化内科, 神经内科, 烧伤整形科, 外二科, 口腔科, 传染科,..."
1630,九台区人民医院,公立,综合医院,二级,NA,"消化内科, 神经外科, 普外科, 妇产科, 传染病, 中医科, 放射科, 病理科, 麻醉科,..."


In [19]:
merged.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-merged.csv",index=False, encoding='utf-8-sig')

In [ ]:
df = pd.read_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\99health-merged.csv")

In [38]:
a = df[df['科室'].isna()]
b= a[a['分级'].isna()]
c = b[b['分等'].isna()]
d = c[c['经营模式'].isna()]
e = d[d['类别'].isna()]
e.index

KeyError: '科室'

In [ ]:
#new_df = df.loc[[i not in e.index for i in np.arange(7083)]]
#new_df